# preparação

## importações

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM, GRU
import pandas as pd
import numpy as np

2022-03-24 14:38:59.452189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 14:38:59.452287: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


CPU times: user 13.7 s, sys: 4.52 s, total: 18.2 s
Wall time: 17.1 s


## Aquisição de dados

In [ ]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data(0)
df

## visualização

In [ ]:
len(df.columns)

In [ ]:
df.head(2).to_dict()

In [ ]:
df.isnull().sum()

## Funções de Amostragem

In [ ]:
def subsample_sequence(df, length):
    '''returns a random sample sequence from inside a dataframe'''
    index = np.random.randint(0, df.shape[0] - length)
    df_sample = df.iloc[index:index+length]
    
    return df_sample

In [ ]:
df_sample = subsample_sequence(df,48)
df_sample.head()

In [ ]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample['Chuva'].iloc[length - 24:]
    
    X_sample = df_subsample.drop(columns= ['classe_chuva', 'datahora'])[0:length -24]
    X_sample = X_sample.values
   
    return np.array(X_sample), np.array(y_sample)

In [ ]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

## Train-test split

In [ ]:
train_size = int(df.shape[0]*0.8)
df_train = df.iloc[:train_size,:]
df_test = df.iloc[train_size:,:]

In [ ]:
df_train.value_counts(['Chuva'])

In [ ]:
df_test.shape

In [ ]:
X_train,y_train = get_X_y(df_train,200,72)

In [ ]:
X_test,y_test = get_X_y(df_test,200,72)

In [ ]:
y_train.shape

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical

# # label = LabelEncoder()
# # y_enc = label.fit_transform(y.reshape((-1,)))
# y_cat = to_categorical(y_train)
# y_cat.shape

## normalização

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

norm = Normalization()
norm.adapt(X_train)

In [ ]:
X_train.shape

# Modelagem

## Arquitetura da rede

In [ ]:
model1 = Sequential()

model1.add(norm)

model1.add(LSTM(units=100, activation='tanh',return_sequences=False))
model1.add(Dense(80, activation="tanh"))
model1.add(Dense(50, activation="tanh"))
model1.add(Dense(24, activation="relu"))

model1.compile(loss='mape',
                  optimizer='rmsprop',
                  metrics=['mae','mse','mape'])

model1.summary()

In [ ]:
model2 = Sequential()

model2.add(norm)

model2.add(LSTM(units=100, activation='tanh',return_sequences=True))
model2.add(LSTM(units=50, activation='tanh',return_sequences=False))
model2.add(Dense(30, activation="tanh"))
model2.add(Dense(24, activation="relu"))

model2.compile(loss='mape',
                  optimizer='rmsprop',
                  metrics=['mae','mse','mape'])

model2.summary()

In [ ]:
model3 = Sequential()

model3.add(norm)

model2.add(LSTM(units=100, activation='tanh',return_sequences=True))
model2.add(LSTM(units=80, activation='tanh',return_sequences=False))
model2.add(Dense(60, activation="tanh"))
model2.add(Dense(40, activation="tanh"))
model2.add(Dense(24, activation="relu"))

model3.compile(loss='mape',
                  optimizer='rmsprop',
                  metrics=['mae','mse','mape'])

model3.summary()

## ajuste

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
%%time
model1.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

In [ ]:
%%time
model2.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

In [ ]:
%%time
model3.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

# Avaliação

## Tratamento de resultado

In [ ]:
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)
y_pred3 = model3.predict(X_test)

In [ ]:
model1.evaluate(X_test,y_test)
model2.evaluate(X_test,y_test)
model3.evaluate(X_test,y_test)

In [ ]:
df_y_test = pd.DataFrame(y_test).T #cada coluna sera o indice para os dias do futuro -6000 dias

In [ ]:
df_y_pred1 = pd.DataFrame(y_pred1).T
df_y_pred2 = pd.DataFrame(y_pred2).T
df_y_pred3 = pd.DataFrame(y_pred3).T

In [ ]:
pd.DataFrame(y_pred1).describe()
pd.DataFrame(y_pred2).describe()
pd.DataFrame(y_pred3).describe()

Função de categorização

In [ ]:
def classe_chuva(precipitacao):
    mm=precipitacao
    if np.isnan(mm):
        chuva = "NaN"
    if mm == 0:
        chuva = 0 #'nao chove'
    elif mm >0 and mm <=5.0:
        chuva = 1 #'fraca'
    elif mm >5.0 and mm<=25.0:
        chuva = 2 #'moderada'
    else:
        chuva = 3 #'forte'
    return chuva

dataframe com o y test

In [ ]:
df_y_test_categorical = df_y_test.applymap(classe_chuva)

categorizando as previsões

In [ ]:
df_y_pred_categorical1 = df_y_pred1.applymap(classe_chuva)
df_y_pred_categorical2 = df_y_pred2.applymap(classe_chuva)
df_y_pred_categorical3 = df_y_pred3.applymap(classe_chuva)

In [ ]:
(df_y_test_categorical == df_y_pred_categorical1).sum() # o quanto acertamos por dia

In [ ]:
m1acc = (df_y_test_categorical == df_y_pred_categorical1).sum().sum()/(6000*24)
m2acc = (df_y_test_categorical == df_y_pred_categorical2).sum().sum()/(6000*24)
m3acc = (df_y_test_categorical == df_y_pred_categorical3).sum().sum()/(6000*24)

## Avaliação

In [ ]:
print(f'model 1 accuracy: {m1acc}')
print(f'model 2 accuracy: {m2acc}')
print(f'model 3 accuracy: {m3acc}')

In [ ]:
model1.evaluate(X_test,y_test)

In [ ]:
model2.evaluate(X_test,y_test)

In [ ]:
model3.evaluate(X_test,y_test)